# MemMachine V2 Complete Guide

This notebook demonstrates how to use the MemMachine V2 REST API to:
1. Create and manage projects
2. Store memories (episodic and semantic)
3. Search and retrieve memories
4. Delete memories and projects

**Key Concepts:**
- **Projects**: Isolated memory spaces for different applications or users
- **Episodic Memory**: Time-stamped event-based memories (conversations, interactions)
- **Semantic Memory**: Persistent facts and knowledge (user profiles, preferences)


Before you run this notebook, please make sure that MemMachine V2 is running and the endpoints are accessible.

To learn more about the API endpoints, visit `http://localhost:8080/docs`


## Install Required Dependencies

First, let's install the `requests` library. If you're running this in a virtual environment or Jupyter notebook, you may need to restart the kernel after installation.


In [ ]:
import sys
import subprocess

# Install requests package
subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])

## Import Required Libraries

**Note:** If you get a ModuleNotFoundError, please restart your kernel (Kernel -> Restart Kernel) and run the cells again.

We'll use the `requests` library to make HTTP requests to the MemMachine V2 REST API.


In [ ]:
import requests
from datetime import datetime, timezone
import json
print("All libraries imported successfully!")

## Configuration

Set up the base URL for your MemMachine V2 server. The default port is 8080.

### Understanding the Base URL

The base URL structure is:
```
http://{host}:{port}/api/v2
```

Where:
- **host**: Server location (localhost for local development, or IP/domain for remote servers)
- **port**: Default is 8080 for MemMachine V2
- **/api/v2**: API version prefix (V2 uses this prefix for all endpoints)

You can modify this if your MemMachine instance runs on a different port or host.

In [ ]:
BASE_URL = "http://localhost:8080/api/v2"
print(f"API Base URL: {BASE_URL}")

## Step 1: Create a Project

Projects provide isolated memory spaces. Let's create a project for a general chatbot application.

### Why Projects?

Projects are essential for:
- **Data Isolation**: Separate memories for different users or applications
- **Multi-tenancy**: Support multiple organizations or teams
- **Access Control**: Manage who can access what memories
- **Configuration**: Different embedding models or settings per project

### Project Structure

Each project requires:
1. **org_id**: Organization identifier (use a consistent name for your org)
2. **project_id**: Unique project name (lowercase, no spaces recommended)
3. **description**: Human-readable explanation of the project's purpose


In [ ]:
# Define project identifiers
org_id = "demo_org"
project_id = "chatbot_assistant"

print(f"Organization ID: {org_id}")
print(f"Project ID: {project_id}")

In [ ]:
# Create project payload
create_project_payload = {
    "org_id": org_id,
    "project_id": project_id,
    "description": "General purpose chatbot assistant with memory"
}

print("Project configuration:")
print(json.dumps(create_project_payload, indent=2))

In [ ]:
# Create the project
response = requests.post(
    f"{BASE_URL}/projects",
    json=create_project_payload
)

if response.status_code == 201:
    print("Project created successfully!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error creating project: {response.status_code}")
    print(response.text)

## Step 2: Add Semantic Memories (User Profile)

Semantic memories store persistent facts about users, preferences, and long-term knowledge.

### What are Semantic Memories?

Semantic memories are **timeless facts** that don't change frequently:
- User profile information (name, occupation, interests)
- Preferences and settings
- Domain knowledge and facts
- Relationships and connections

### Message Structure

Each semantic memory message contains:
- **content**: The actual information to store
- **producer**: Who created this memory (usually "user" or "system")
- **produced_for**: Who this memory is about or for (e.g., "assistant")
- **timestamp**: When this memory was created (ISO format)
- **metadata**: Additional contextual information as key-value pairs

In [ ]:
# Add user profile information
user_id = "Alex1"

semantic_payload = {
    "org_id": org_id,
    "project_id": project_id,
    "types": ["semantic"],
    "messages": [
        {
            "content": "My name is Alex Johnson",
            "producer": user_id,
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "metadata": {
                "category": "personal_info",
                "type": "name",
                "user_id": user_id
            }
        },
        {
            "content": "I am a software engineer specializing in machine learning",
            "producer": user_id,
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "metadata": {
                "category": "professional_info",
                "type": "occupation",
                "user_id": user_id
            }
        },
        {
            "content": "I prefer Python programming language and enjoys hiking on weekends",
            "producer": user_id,
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "metadata": {
                "category": "interests",
                "type": "preferences",
                "user_id": user_id
            }
        },
        {
            "content": "My sister lives in Mumbai and is a doctor.",
            "producer": user_id,
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "metadata": {
                "category": "family",
                "user_id": user_id
            }
        },
        {
            "content": "I have an orange cat named Appa and a husky named Momo",
            "producer": user_id,
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "metadata": {
                "category": "pets",
                "user_id": user_id
            }
        }
    ]
}

response = requests.post(
    f"{BASE_URL}/memories",
    json=semantic_payload
)

if response.status_code == 200:
    print("Semantic memories added successfully!")
    result = response.json()
    print(f"Added {len(result.get('results', []))} memories")
else:
    print(f"Error adding semantic memories: {response.status_code}")
    print(response.text)

## Step 3: Add Episodic Memories (Conversation History)

Episodic memories store time-stamped conversations and interactions.

### What are Episodic Memories?

Episodic memories capture **specific events and interactions** over time.

### Key Differences from Semantic Memory

| Aspect | Episodic | Semantic |
|--------|----------|----------|
| **Time** | Time-stamped events | Timeless facts |
| **Change** | Frequently added | Rarely changed |
| **Structure** | Sequential conversations | Isolated facts |
| **Use Case** | "What did we discuss?" | "What do I know about the user?" |

In [ ]:
# Simulate a conversation
conversation = [
    {"role": "user", "content": "Hi! Can you help me with a Python programming question?"},
    {"role": "assistant", "content": "Of course! I'd be happy to help. What's your question?"},
    {"role": "user", "content": "How do I read a CSV file using pandas?"},
    {"role": "assistant", "content": "You can use pd.read_csv('filename.csv'). Would you like more details?"},
    {"role": "user", "content": "Thanks! That's exactly what I needed."},
]

print(f"Adding conversation with {len(conversation)} messages...")

In [ ]:
# Add conversation to episodic memory
episodic_payload = {
    "org_id": org_id,
    "project_id": project_id,
    "types": ["episodic"],
    "messages": [
        {
            "content": msg["content"],
            "producer": msg["role"],
            "produced_for": "assistant" if msg["role"] == "user" else "user",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "role": msg["role"],
            "metadata": {
                "conversation_id": "conv_001",
                "topic": "python_programming",
                "user_id": user_id
            }
        }
        for msg in conversation
    ]
}

response = requests.post(
    f"{BASE_URL}/memories",
    json=episodic_payload
)

if response.status_code == 200:
    print("Episodic memories added successfully!")
    result = response.json()
    print(f"Added {len(result.get('results', []))} memories")
else:
    print(f"Error adding episodic memories: {response.status_code}")
    print(response.text)

## Step 4: Add More Varied Memories

Let's add some diverse memories covering different topics.

In [ ]:
# Add more episodic memories
varied_messages = [
    "I'm planning a trip to Tokyo next month",
    "Need to prepare a presentation on neural networks for the team meeting",
    "Found a great new coffee shop downtown called Blue Bottle",
    "Working on implementing a recommendation system using collaborative filtering",
    "Completed the machine learning course on Coursera last week",
    "Planning to learn React for my next project",
    "Had a productive discussion about API design patterns",
]

print(f"Adding {len(varied_messages)} additional memories...\n")

for i, message in enumerate(varied_messages, 1):
    payload = {
        "org_id": org_id,
        "project_id": project_id,
        "types": ["episodic"],
        "messages": [{
            "content": message,
            "producer": user_id,
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "role": "user",
            "metadata": {
                "user_id": user_id
            }
        }]
    }
    response = requests.post(
        f"{BASE_URL}/memories",
        json=payload
    )
    status = "✓" if response.status_code == 200 else "✗"
    print(f"{status} [{i}/{len(varied_messages)}] {message[:60]}...")

print("\nAll memories added!")

## Step 5: Search Memories

Now let's search for relevant memories using semantic search.

### How Semantic Search Works

MemMachine uses **vector embeddings** for semantic search:
1. Your query is converted to a vector representation
2. All stored memories are also represented as vectors
3. The system finds memories with similar vectors (similar meaning)
4. Results are ranked by relevance and sorted by timestamp

### Search Parameters

- **query**: Natural language search query (what you're looking for)
- **top_k**: Number of results to return
- **types**: Which memory types to search
  - `["episodic"]`: Search only conversations
  - `["semantic"]`: Search only facts/profile
  - `["episodic", "semantic"]`: Search both (default)
- **filter**: Optional metadata filtering

In [ ]:
# Search for programming-related memories
search_payload = {
    "org_id": org_id,
    "project_id": project_id,
    "top_k": 5,
    "query": "Python programming and machine learning",
    "types": ["episodic", "semantic"],
}

print("Searching for: Python programming and machine learning\n")

response = requests.post(
    f"{BASE_URL}/memories/search",
    json=search_payload
)

if response.status_code == 200:
    print("✓ Search completed successfully!")
    print("\nSearch results:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"✗ Error searching memories: {response.status_code}")
    print(response.text)

In [ ]:
# Search for travel-related memories
search_payload["query"] = "travel plans and trips"

print("Searching for: travel plans and trips\n")

response = requests.post(
    f"{BASE_URL}/memories/search",
    json=search_payload
)

if response.status_code == 200:
    print("Search completed successfully!")
    print("\nTravel search results:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error searching memories: {response.status_code}")
    print(response.text)

In [ ]:
# Search only semantic memory (user profile)
search_payload["query"] = "What is the user's name?"
search_payload["types"] = ["semantic"]

print("Searching semantic memory for: user profile information\n")

response = requests.post(
    f"{BASE_URL}/memories/search",
    json=search_payload
)

if response.status_code == 200:
    print("Search completed successfully!")
    print("\nProfile search results:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error searching memories: {response.status_code}")
    print(response.text)

## Step 6: List All Memories

Retrieve all memories in the project with pagination.

In [ ]:
# List episodic memories
list_payload = {
    "org_id": org_id,
    "project_id": project_id,
    "page_size": 10,
    "page_num": 0,
    "filter": "",
    "type": "episodic"
}

print("Listing episodic memories...\n")

response = requests.post(
    f"{BASE_URL}/memories/list",
    json=list_payload
)

if response.status_code == 200:
    print("Episodic memories retrieved successfully!")
    print("\nEpisodic memories list:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error listing memories: {response.status_code}")
    print(response.text)

In [ ]:
# List semantic memories
list_payload["type"] = "semantic"

print("Listing semantic memories...\n")

response = requests.post(
    f"{BASE_URL}/memories/list",
    json=list_payload
)

if response.status_code == 200:
    print("Semantic memories retrieved successfully!")
    print("\nSemantic memories list:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error listing memories: {response.status_code}")
    print(response.text)

## Step 7: Get Episode Count

Check how many episodic memories are stored in the project.

In [ ]:
count_payload = {
    "org_id": org_id,
    "project_id": project_id
}

response = requests.post(
    f"{BASE_URL}/projects/episode_count/get",
    json=count_payload
)

if response.status_code == 200:
    count = response.json().get('count', 0)
    print(f"Episode count: {count}")
else:
    print(f"Error getting episode count: {response.status_code}")
    print(response.text)

## Step 8: Get Project Details

Retrieve information about the project.

In [ ]:
get_project_payload = {
    "org_id": org_id,
    "project_id": project_id
}

response = requests.post(
    f"{BASE_URL}/projects/get",
    json=get_project_payload
)

if response.status_code == 200:
    print("Project details retrieved successfully!")
    print("\nProject details:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error getting project: {response.status_code}")
    print(response.text)

## Step 9: List All Projects

Get a list of all projects in the organization.

In [ ]:
response = requests.post(f"{BASE_URL}/projects/list")

if response.status_code == 200:
    print("✓ Projects list retrieved successfully!")
    print("\nAll projects:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"✗ Error listing projects: {response.status_code}")
    print(response.text)

## Step 10: Health Check

Verify that the MemMachine V2 server is running properly.

In [ ]:
response = requests.get(f"{BASE_URL}/health")

if response.status_code == 200:
    print("✓ Server is healthy!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"✗ Health check failed: {response.status_code}")
    print(response.text)

## Advanced Example: Multi-User Scenario

Let's create separate projects for different users to demonstrate project isolation.

### Why Project Isolation Matters

In production applications, you need to:
- **Protect Privacy**: Keep each user's memories separate
- **Prevent Data Leakage**: User A shouldn't see User B's conversations
- **Scale Independently**: Different users can have different storage needs
- **Custom Configuration**: Each project can use different embedding models

In [ ]:
# Create projects for different users
users = [
    {"org_id": "demo_org", "project_id": "user_alice", "name": "Alice", "role": "Data Scientist"},
    {"org_id": "demo_org", "project_id": "user_bob", "name": "Bob", "role": "Product Manager"},
    {"org_id": "demo_org", "project_id": "user_carol", "name": "Carol", "role": "UX Designer"}
]

print(f"Creating projects for {len(users)} users...\n")

for user in users:
    # Create project
    project_payload = {
        "org_id": user["org_id"],
        "project_id": user["project_id"],
        "description": f"Memory space for {user['name']}",
        "config": {
            "embedder": "openai_embedder",
            "reranker": "my_reranker_id"
        }
    }
    response = requests.post(f"{BASE_URL}/projects", json=project_payload)
    status = "✓" if response.status_code == 201 else "✗"
    print(f"{status} Created project for {user['name']}")
    
    # Add user profile
    semantic_payload = {
        "org_id": user["org_id"],
        "project_id": user["project_id"],
        "types": ["semantic"],
        "messages": [{
            "content": f"User name is {user['name']} and works as a {user['role']}",
            "producer": "system",
            "produced_for": "assistant",
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "metadata": {"user_id": user["project_id"]}
        }]
    }
    response = requests.post(f"{BASE_URL}/memories", json=semantic_payload)
    status = "✓" if response.status_code == 200 else "✗"
    print(f"{status} Added profile for {user['name']}\n")

print("✓ All user projects created!")

## Cleanup (Optional)

Delete specific memories or entire projects when needed.

**Warning**: Deletion operations are permanent and cannot be undone!

In [ ]:
# Note: Uncomment to delete
# delete_payload = {
#     "org_id": org_id,
#     "project_id": project_id
# }
# response = requests.post(f"{BASE_URL}/projects/delete", json=delete_payload)
# if response.status_code == 204:
#     print(f"✓ Project '{project_id}' deleted successfully")
# else:
#     print(f"✗ Error deleting project: {response.status_code}")
#     print(response.text)

print("Cleanup section (commented out for safety)")

## Summary

This notebook demonstrated the complete workflow for using MemMachine V2:

### What We Covered

1. **Project Management**
   - Creating isolated projects for different users/applications
   - Managing project lifecycle

2. **Memory Storage**
   - **Semantic Memory**: Long-term facts and user profiles
   - **Episodic Memory**: Time-stamped conversations and events
   - Proper message structure and metadata

3. **Memory Retrieval**
   - Semantic search across memory types
   - Filtering by memory type
   - Pagination and listing

4. **Multi-User Support**
   - Project isolation for privacy
   - Multi-tenant architecture
   - Scalable user management

5. **System Monitoring**
   - Health checks
   - Episode counting
   - Project listing

### Key Takeaways

**Projects provide complete isolation:**
- Essential for multi-user applications
- Each project has independent memory stores

**Choose the right memory type:**
- **Episodic**: For "What happened?" questions (conversations, events)
- **Semantic**: For "What do I know?" questions (facts, profiles)

**Search is powerful:**
- Vector-based semantic search finds relevant content
- Can search across or within specific memory types
- Natural language queries work best
- Results sorted by timestamp for context

**Metadata is your friend:**
- Use metadata to organize memories
- Enables filtering during search and list operations
- Helps with future retrieval

### Next Steps

Now that you understand the basics, you can:

1. **Build Your Application**
   - Integrate MemMachine into your chatbot or assistant
   - Implement conversation history
   - Store user preferences

2. **Optimize Performance**
   - Tune the number of search results (top_k)
   - Experiment with different query formulations
   - Monitor memory usage and episode counts

3. **Scale Your System**
   - Create projects dynamically for new users
   - Implement proper error handling
   - Add authentication and authorization

4. **Advanced Features**
   - Implement memory cleanup strategies
   - Build analytics on stored memories
   - Create custom metadata schemas
   - Use metadata filters in search and list operations

### Resources

- **API Documentation**: `http://localhost:8080/docs`
- **OpenAPI Spec**: `http://localhost:8080/openapi.json`
- **MemVerge Website**: For additional resources and support

### Troubleshooting

**Common Issues:**

1. **Connection Refused**: Make sure MemMachine is running on port 8080
2. **422 Validation Error**: Check your request payload matches the schema
3. **Empty Search Results**: Ensure memories were stored successfully
4. **Slow Responses**: Large result sets or complex queries may take time

For more information, visit the API documentation at `http://localhost:8080/docs`